In [41]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np
import statistics

import math

fc = 1
#na = np.empty((1, 51))
na = []
actualKp = []
currentKeypoints = []
c=0
lK = []
nK = []

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [43]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
        image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
        output, _ = model(image)
    return output, image

In [44]:
def draw_keypoints(output, image):
    global fc, c, t, nK, lK, na, nc
    output = non_max_suppression_kpt(output, 
                                     0.08, # Confidence Threshold
                                     0.2, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    #0.2, 0.4
    with torch.no_grad():
        output = output_to_keypoint(output)
        #print(f'Frame Number: {fc}; Data Size: {output.shape}')
    try:
        t = output[0] #retrieves only first skeleton data
        t = t[-51:] #retrieves last 51 elements
        #t = t[::3] cuts every third element (confidence level)
        t = [x for i, x in enumerate(t) if (i+1)%3 != 0]
        #append all nose x coords
        nK.append(t[2])
        #appends all hip x coords
        lK.append(t[22]) #array t is unsorted, the x values is every other starting at index 0 and y values every other starting at 1
        #index 22 gives the x coordinate for right hip joint
        na.append(t)
        currentKeypoints = []
    except:
        #currentKeypoints = 
        c += 1
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    
    
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    
    
        
    return nimg

In [45]:
nc = 0
fa = []
def swimPose_estimate(filename, savepath):
    global fc, c, t, nK, lK, na, fa
    
    cap = cv2.VideoCapture(filename)
    totalFrames = math.floor(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))/100)
    print(f'TF: {totalFrames}')
    i = 0
    fa = []
    
    while i < totalFrames:
        na = []
        fc = 0
        nK = []
        lK = []
        c=0
        poseH(filename, "none", i*100)
        print(f'Original data: {c} empty frames')
        #print("nK: ",nK)
        #print("lK: ",lK)

        #print("-=-=-=-=-=-=-=-")
        #print("nK mode: ",statistics.median(nK))
        #print("lK mode: ",statistics.median(lK))
        #print(c)
        cap.release()
        cv2.destroyAllWindows()
        
        if c > 10:
            fc = 0
            c=0
            if statistics.median(nK) > statistics.median(lK):
                na = []
                #print("needs counterclockwise rotation")
                poseH(filename, "cc", i*100)
                print("transformation completed")
                print(f'Counterclockwise rotation; {c} empty frames')
                if c <= 20:
                    fa.extend(na)
                else:
                    print("too many missing frames, batch discarded")
            else:
                na = []
                #print("needs clockwise rotation")
                poseH(filename, "c", i*100)
                print("transformation completed")
                print(f'Clockwise rotation; {c} empty frames')
                if c <= 25:
                    fa.extend(na)
                else:
                    print("too many missing frames, batch discarded")
        else:
            print("no change necessary")
            fa.extend(na)

        
        #print(na[10])
        #skelData = np.array(na)
        #skelData = skelData.reshape(skelData.shape[0], 17, 2)
        #print(skelData[10])
    
        i += 1
        print(f'batch {i} complete')
        
    print("=======================================================")
    print("-----------Skeleton Data Extraction Complete-----------")
    print("=======================================================")

    x = np.array(fa)
    x = np.reshape(x, (x.shape[0], 17, 2))
    print(f'Array shape: {x.shape}')
    print(f'Data saved to: {savepath}')
    
    print("=======================================================")
    cv2.destroyAllWindows()

In [46]:
def poseH(filename, rotation, currentFrame):
    global fc, c, t, nK, lK, na
    cv2.destroyAllWindows()

    cap = cv2.VideoCapture(filename)
    # VideoWriter for saving the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('Free_Skel.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, currentFrame)

    
    while fc < 100 and cap.isOpened():
        (ret, frame) = cap.read()
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if rotation == "cc":
                frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
            elif rotation == "c":
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            elif rotation == "none":
                pass

            output, frame = run_inference(frame)
            frame = draw_keypoints(output, frame)
            fc += 1
            
            frame = cv2.resize(frame, (int(cap.get(3)), int(cap.get(4))))
            
            if rotation == "cc" or rotation == "c":
                frame = cv2.resize(frame,(720,1280),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            else:
                frame = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            out.write(frame)
            cv2.imshow('Pose estimation', frame)
        else:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [47]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Free Training Data.mp4"
path = 'Free_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise

TF: 161
Original data: 58 empty frames
transformation completed
Counterclockwise rotation; 5 empty frames
batch 1 complete
Original data: 69 empty frames
transformation completed
Counterclockwise rotation; 0 empty frames
batch 2 complete
Original data: 53 empty frames
transformation completed
Counterclockwise rotation; 0 empty frames
batch 3 complete
Original data: 62 empty frames
transformation completed
Counterclockwise rotation; 0 empty frames
batch 4 complete
Original data: 35 empty frames
transformation completed
Counterclockwise rotation; 1 empty frames
batch 5 complete
Original data: 42 empty frames
transformation completed
Counterclockwise rotation; 10 empty frames
batch 6 complete
Original data: 4 empty frames
no change necessary
batch 7 complete
Original data: 7 empty frames
no change necessary
batch 8 complete
Original data: 49 empty frames
transformation completed
Counterclockwise rotation; 0 empty frames
batch 9 complete
Original data: 50 empty frames
transformation comple

In [ ]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Fly Training Data.mp4"
path = 'Fly_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise

TF: 210
Original data: 7 empty frames
no change necessary
batch 1 complete
Original data: 1 empty frames
no change necessary
batch 2 complete
Original data: 0 empty frames
no change necessary
batch 3 complete
Original data: 2 empty frames
no change necessary
batch 4 complete
Original data: 4 empty frames
no change necessary
batch 5 complete
Original data: 1 empty frames
no change necessary
batch 6 complete
Original data: 5 empty frames
no change necessary
batch 7 complete
Original data: 8 empty frames
no change necessary
batch 8 complete
Original data: 3 empty frames
no change necessary
batch 9 complete
Original data: 0 empty frames
no change necessary
batch 10 complete
Original data: 4 empty frames
no change necessary
batch 11 complete
Original data: 0 empty frames
no change necessary
batch 12 complete
Original data: 11 empty frames
transformation completed
Counterclockwise rotation; 6 empty frames
batch 13 complete
Original data: 19 empty frames
transformation completed
Counterclockw

In [ ]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Back Training Data.mp4"
path = 'Back_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise

In [ ]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Breast Training Data.mp4"
path = 'Breast_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise

In [ ]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Underwater Training Data.mp4"
path = 'Underwater_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise

In [ ]:
%%time
video = "C:/Users/jonso/OneDrive/Desktop/Dive Training Data.mp4"
path = 'Dive_Skel_Training.npy'
swimPose_estimate(video, path)
#counterclockwise